# Understanding Hired Rides in NYC



## Project Setup

In [4]:
# all import statements needed for the project, for example:

import os
import bs4
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import re
from datetime import datetime, timedelta
import numpy as np
import math
import glob

In [5]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "data/weather"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [6]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [12]:
def load_taxi_zones(shapefile):
    """Reading taxi zones shapefile through geopandas."""
    geofile = gpd.read_file(shapfile)
    return geofile

In [14]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
    # Find the taxi zone id corresponding to the location id
    zone = loaded_taxi_zones[loaded_taxi_zones['LocationID'] == zone_loc_id]
    
    # If no match is found, return None
    if zone.empty:
        return None
    
    # Get the centroid of the zone's geometry
    centroid = zone.geometry.centroid.iloc[0]
    
    # Return the latitude and longitude as a tuple
    return (centroid.y, centroid.x)

### Calculate Sample Size

In [17]:
def calculate_sample_size(population: int, confidence_level: float = 0.95, margin_of_error: float = 0.05, p: float = 0.5) -> int:
    """
    Calculate the sample size using Cochran's formula.

    Args:
        population (int): The total number of data points in the population.
        confidence_level (float): The confidence level (default is 0.95).
        margin_of_error (float): The margin of error (default is 0.05).
        p (float): The estimated proportion of the population.

    Returns:
        int: The calculated sample size.
    """
    z = {0.90: 1.645, 0.95: 1.96, 0.99: 2.576}.get(confidence_level, 1.96)
    numerator = (z ** 2) * p * (1 - p)
    denominator = margin_of_error ** 2
    sample_size = numerator / denominator

    if population > 0:
        adjusted_sample_size = sample_size / (1 + (sample_size - 1) / population)
    else:
        adjusted_sample_size = sample_size
    
    return int(np.ceil(adjusted_sample_size))

### Common Functions

In [20]:
def get_all_urls_from_tlc_page(taxi_page):
    """
    Fetches all URLs from a given webpage.

    Args:
        taxi_page (str): URL of the webpage to scrape.

    Returns:
        list: List of all URLs found on the webpage.
    """
    try:
        # Send a GET request to the page
        response = requests.get(taxi_page)
        response.raise_for_status()  # Raise an HTTPError for bad responses
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to access the URL: {taxi_page}. Error: {e}")
    
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all anchor tags with href attributes
    links = soup.find_all("a", href=True)
    
    # Extract and return all href attributes
    all_urls = [link["href"] for link in links]
    
    return all_urls

In [22]:
def filter_parquet_urls(all_urls):
    parquet_urls = []
    for url in all_urls:
        # Normalize the URL (strip whitespace, handle cases like trailing slashes)
        url = url.strip()
        # Use regex to ensure matching even with query parameters
        if re.search(r"\.parquet(\?.*)?$", url):
            parquet_urls.append(url)
    return parquet_urls

### Process Taxi Data

In [25]:
def get_and_clean_taxi_month(url: str) -> pd.DataFrame:
    """
    Downloads, processes, and saves Yellow Taxi dataset for a given month.

    Args:
        url (str): URL of the Yellow Taxi Parquet file.

    Returns:
        pd.DataFrame: Sampled and processed DataFrame.
    """
    # Default directory for processed Yellow Taxi data
    save_dir = "processed_data/yellow_taxi"

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Extract file name and define local path
    file_name = url.split("/")[-1]
    local_file_path = os.path.join(save_dir, file_name)

    # Download the file if not already downloaded
    if not os.path.exists(local_file_path):
        print(f"Downloading Yellow Taxi file: {url} ...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            with open(local_file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1MB chunks
                    if chunk:
                        f.write(chunk)
            print(f"File saved to: {local_file_path}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if download fails
    else:
        print(f"Loading file from local storage: {local_file_path}")

    # Load the dataset
    try:
        data = pd.read_parquet(local_file_path)
    except Exception as e:
        print(f"Error reading Parquet file {local_file_path}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if reading fails

    # Determine population size
    population = len(data)
    print(f"Population size: {population}")

    # Calculate sample size (using p = 0.5 for Yellow Taxi data)
    sample_size = calculate_sample_size(population, p = 0.5)
    print(f"Calculated sample size: {sample_size}")

    # Sample the dataset
    sampled_data = data.sample(n=sample_size, random_state=42) if population > sample_size else data

    # Save the sampled dataset
    processed_file_path = os.path.join(save_dir, f"sampled_{file_name}")
    sampled_data.to_parquet(processed_file_path)
    print(f"Processed file saved to: {processed_file_path}")

    return sampled_data

In [27]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []

    # filter urls of yellow taxi data
    yellow_taxi_pattern = re.compile(r"yellow_tripdata_(2020-(0[1-9]|1[0-2])|202[1-3]-(0[1-9]|1[0-2])|2024-(0[1-8]))\.parquet")
    yellow_taxi_urls = [url for url in parquet_urls if yellow_taxi_pattern.search(url)]
    
    for url in yellow_taxi_urls:
        # clean data through function defined before
        dataframe = get_and_clean_taxi_month(url)

        # solving the problem that dataframes contain different name for airport fee due to uppercase/lowercase difference
        if "Airport_fee" in list(dataframe):
            dataframe.rename(columns={"Airport_fee": "airport_fee"}, inplace=True)
        
        all_taxi_dataframes.append(dataframe)
        
    # create a dataframe combining all taxi data
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [29]:
def get_taxi_data():
    # Applying functions defined to get the taxi data
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = filter_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [31]:
taxi_data = get_taxi_data()

Loading file from local storage: processed_data/yellow_taxi/yellow_tripdata_2024-01.parquet
Population size: 2964624
Calculated sample size: 385
Processed file saved to: processed_data/yellow_taxi/sampled_yellow_tripdata_2024-01.parquet
Loading file from local storage: processed_data/yellow_taxi/yellow_tripdata_2024-02.parquet
Population size: 3007526
Calculated sample size: 385
Processed file saved to: processed_data/yellow_taxi/sampled_yellow_tripdata_2024-02.parquet
Loading file from local storage: processed_data/yellow_taxi/yellow_tripdata_2024-03.parquet
Population size: 3582628
Calculated sample size: 385
Processed file saved to: processed_data/yellow_taxi/sampled_yellow_tripdata_2024-03.parquet
Loading file from local storage: processed_data/yellow_taxi/yellow_tripdata_2024-04.parquet
Population size: 3514289
Calculated sample size: 385
Processed file saved to: processed_data/yellow_taxi/sampled_yellow_tripdata_2024-04.parquet
Loading file from local storage: processed_data/yell

/var/folders/kg/5js8flcn3c138fq2kf35s9vh0000gn/T/ipykernel_70862/3668426633.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxi_data = pd.concat(all_taxi_dataframes)


In [33]:
taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1725696,2,2024-01-20 13:31:30,2024-01-20 14:03:25,2.0,17.14,2.0,N,132,233,1,70.0,0.0,0.5,8.27,6.94,1.0,90.96,2.5,1.75
1581136,2,2024-01-18 21:52:46,2024-01-18 22:03:21,1.0,2.49,1.0,N,163,75,1,13.5,1.0,0.5,4.00,0.00,1.0,22.50,2.5,0.00
19137,2,2024-01-01 03:43:58,2024-01-01 03:50:47,2.0,1.84,1.0,N,127,20,2,10.0,1.0,0.5,0.00,0.00,1.0,12.50,0.0,0.00
1682810,1,2024-01-19 22:20:12,2024-01-19 22:50:12,1.0,3.60,1.0,N,186,263,1,23.3,3.5,0.5,5.65,0.00,1.0,33.95,2.5,0.00
511035,2,2024-01-06 22:41:50,2024-01-06 22:43:24,1.0,0.04,1.0,N,238,238,2,3.7,1.0,0.5,0.00,0.00,1.0,6.20,0.0,0.00


In [35]:
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21556 entries, 1725696 to 701728
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               21556 non-null  int64         
 1   tpep_pickup_datetime   21556 non-null  datetime64[us]
 2   tpep_dropoff_datetime  21556 non-null  datetime64[us]
 3   passenger_count        20381 non-null  float64       
 4   trip_distance          21556 non-null  float64       
 5   RatecodeID             20381 non-null  float64       
 6   store_and_fwd_flag     20381 non-null  object        
 7   PULocationID           21556 non-null  int64         
 8   DOLocationID           21556 non-null  int64         
 9   payment_type           21556 non-null  int64         
 10  fare_amount            21556 non-null  float64       
 11  extra                  21556 non-null  float64       
 12  mta_tax                21556 non-null  float64       
 13 

In [37]:
taxi_data.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,21556.000000,21556,21556,20381.000000,21556.000000,20381.000000,21556.000000,21556.000000,21556.000000,21556.000000,21556.000000,21556.000000,21556.000000,21556.000000,21556.000000,21556.000000,20381.000000,15054.000000
mean,1.719197,2022-05-01 19:16:50.909074,2022-05-01 19:33:07.142048,1.399980,3.292046,1.468132,165.086472,161.552282,1.184682,15.582768,1.192449,0.488194,2.692410,0.451578,0.542666,22.609807,2.265958,0.113442
min,1.000000,2009-01-01 01:11:17,2009-01-01 01:11:20,0.000000,0.000000,1.000000,4.000000,1.000000,0.000000,-250.000000,-7.500000,-0.500000,-0.130000,-34.200000,-1.000000,-251.000000,-2.500000,-1.750000
25%,1.000000,2021-03-01 12:26:32.750000,2021-03-01 12:34:07,1.000000,1.050000,1.000000,132.000000,107.000000,1.000000,7.200000,0.000000,0.500000,0.000000,0.000000,0.300000,12.600000,2.500000,0.000000
50%,2.000000,2022-05-01 13:35:11.500000,2022-05-01 14:01:36.500000,1.000000,1.800000,1.000000,162.000000,162.000000,1.000000,10.700000,0.500000,0.500000,2.150000,0.000000,0.300000,17.020000,2.500000,0.000000
75%,2.000000,2023-07-01 00:03:05,2023-07-01 00:12:55.500000,1.000000,3.310000,1.000000,234.000000,234.000000,1.000000,17.500000,2.500000,0.500000,3.440000,0.000000,1.000000,24.800000,2.500000,0.000000
max,6.000000,2024-08-31 23:00:33,2024-08-31 23:34:12,6.000000,67.900000,99.000000,265.000000,265.000000,4.000000,278.800000,11.750000,0.500000,51.150000,57.050000,1.000000,289.350000,2.500000,1.750000
std,0.493676,NaN,NaN,0.981261,4.272702,6.286294,65.699489,70.905294,0.568066,15.408238,1.511804,0.097580,3.282479,1.909912,0.353155,19.130045,0.780689,0.405524


### Processing Uber Data

In [40]:
def get_and_clean_uber_month(url):
    save_dir = "processed_data/hvhf"

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Extract file name and define local path
    file_name = url.split("/")[-1]
    local_file_path = os.path.join(save_dir, file_name)

    # Download the file if not already downloaded
    if not os.path.exists(local_file_path):
        print(f"Downloading HVHF file: {url} ...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            with open(local_file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1MB chunks
                    if chunk:
                        f.write(chunk)
            print(f"File saved to: {local_file_path}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if download fails
    else:
        print(f"Loading file from local storage: {local_file_path}")

    # Load the dataset
    try:
        data = pd.read_parquet(local_file_path)
    except Exception as e:
        print(f"Error reading Parquet file {local_file_path}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if reading fails

    # Determine population size
    population = len(data)
    print(f"Population size: {population}")

    # Calculate sample size (using p = 0.5 for Yellow Taxi data)
    sample_size = calculate_sample_size(population, p = 0.4)
    print(f"Calculated sample size: {sample_size}")

    # Sample the dataset
    sampled_data = data.sample(n=sample_size, random_state=42) if population > sample_size else data

    # Save the sampled dataset
    processed_file_path = os.path.join(save_dir, f"sampled_{file_name}")
    sampled_data.to_parquet(processed_file_path)
    print(f"Processed file saved to: {processed_file_path}")

    return sampled_data

In [42]:
def get_and_clean_uber_data(parquet_urls):
    all_uber_dataframes = []

    # filter urls of hvfhv data
    hvfhv_pattern = re.compile(r"fhvhv_tripdata_(2020-(0[1-9]|1[0-2])|202[1-3]-(0[1-9]|1[0-2])|2024-(0[1-8]))\.parquet")
    hvfhv_urls = [url for url in parquet_urls if hvfhv_pattern.search(url)]
    
    for url in hvfhv_urls:
        # clean data through function defined before
        dataframe = get_and_clean_uber_month(url)
        
        all_uber_dataframes.append(dataframe)
        
    # create a dataframe combining all hvfhv data
    uber_data = pd.concat(all_uber_dataframes)
    return uber_data

In [44]:
def load_and_clean_uber_data(uber_data):
    uber_data['hvfhs_license_num'] = uber_data['hvfhs_license_num'].astype(str)
    
    # Filter rows where 'hvfhs_license_num' is 'HV0003' (Uber)
    uber_only_data = uber_data[uber_data['hvfhs_license_num'] == 'HV0003'].copy()
    return uber_only_data

In [46]:
def get_uber_data():
    # Applying functions defined to get uber data
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = filter_parquet_urls(all_urls)
    u_data = get_and_clean_uber_data(all_parquet_urls)
    uber_data = load_and_clean_uber_data(u_data)
    return uber_data

In [48]:
uber_data = get_uber_data()

Loading file from local storage: processed_data/hvhf/fhvhv_tripdata_2024-01.parquet
Population size: 19663930
Calculated sample size: 369
Processed file saved to: processed_data/hvhf/sampled_fhvhv_tripdata_2024-01.parquet
Loading file from local storage: processed_data/hvhf/fhvhv_tripdata_2024-02.parquet
Population size: 19359148
Calculated sample size: 369
Processed file saved to: processed_data/hvhf/sampled_fhvhv_tripdata_2024-02.parquet
Loading file from local storage: processed_data/hvhf/fhvhv_tripdata_2024-03.parquet
Population size: 21280788
Calculated sample size: 369
Processed file saved to: processed_data/hvhf/sampled_fhvhv_tripdata_2024-03.parquet
Loading file from local storage: processed_data/hvhf/fhvhv_tripdata_2024-04.parquet
Population size: 19733038
Calculated sample size: 369
Processed file saved to: processed_data/hvhf/sampled_fhvhv_tripdata_2024-04.parquet
Loading file from local storage: processed_data/hvhf/fhvhv_tripdata_2024-05.parquet
Population size: 20704538
Ca

/var/folders/kg/5js8flcn3c138fq2kf35s9vh0000gn/T/ipykernel_70862/3445468452.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  uber_data = pd.concat(all_uber_dataframes)


In [50]:
uber_data.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
15895061,HV0003,B03404,B03404,2024-01-26 07:58:05,2024-01-26 08:06:17,2024-01-26 08:07:17,2024-01-26 08:35:38,85,77,4.29,...,2.44,0.00,0.0,0.00,24.88,N,N,N,N,N
11113736,HV0003,B03404,B03404,2024-01-19 02:12:56,2024-01-19 02:16:27,2024-01-19 02:17:05,2024-01-19 02:29:12,220,243,2.55,...,1.34,0.00,0.0,0.00,10.19,N,N,N,N,N
12716273,HV0003,B03404,B03404,2024-01-21 01:39:37,2024-01-21 01:43:40,2024-01-21 01:44:00,2024-01-21 02:08:30,164,80,6.37,...,2.18,2.75,0.0,3.01,23.00,N,N,N,N,N
12099486,HV0003,B03404,B03404,2024-01-20 12:56:18,2024-01-20 12:58:28,2024-01-20 12:58:40,2024-01-20 13:15:42,161,246,1.99,...,1.68,2.75,0.0,0.00,12.47,N,N,N,N,N
730768,HV0003,B03404,B03404,2024-01-02 08:22:37,2024-01-02 08:40:29,2024-01-02 08:40:48,2024-01-02 08:54:28,77,177,2.23,...,1.43,0.00,0.0,0.00,10.64,N,N,N,N,N


In [52]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14735 entries, 15895061 to 5156077
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     14735 non-null  object        
 1   dispatching_base_num  14735 non-null  object        
 2   originating_base_num  14732 non-null  object        
 3   request_datetime      14735 non-null  datetime64[us]
 4   on_scene_datetime     14735 non-null  datetime64[us]
 5   pickup_datetime       14735 non-null  datetime64[us]
 6   dropoff_datetime      14735 non-null  datetime64[us]
 7   PULocationID          14735 non-null  int64         
 8   DOLocationID          14735 non-null  int64         
 9   trip_miles            14735 non-null  float64       
 10  trip_time             14735 non-null  int64         
 11  base_passenger_fare   14735 non-null  float64       
 12  tolls                 14735 non-null  float64       
 13  bcf         

In [54]:
uber_data.describe()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay
count,14735,14735,14735,14735,14735.000000,14735.000000,14735.000000,14735.000000,14735.000000,14735.000000,14735.000000,14735.000000,14735.000000,10782.000000,14735.000000,14735.000000
mean,2022-05-10 11:24:19.137292,2022-05-10 11:27:55.282049,2022-05-10 11:29:03.377264,2022-05-10 11:47:31.430879,137.995521,141.152969,4.850448,1108.307567,22.545591,1.039011,0.668585,1.893382,1.018409,0.190248,0.905722,18.016880
min,2020-01-01 02:45:20,2020-01-01 02:48:30,2020-01-01 02:50:21,2020-01-01 03:12:01,3.000000,1.000000,0.000000,47.000000,-24.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2021-02-25 15:22:55,2021-02-25 15:26:33,2021-02-25 15:27:01,2021-02-25 15:50:44,74.000000,74.000000,1.580000,568.000000,10.660000,0.000000,0.290000,0.890000,0.000000,0.000000,0.000000,8.460000
50%,2022-05-21 20:05:55,2022-05-21 20:18:20,2022-05-21 20:19:41,2022-05-21 20:45:14,140.000000,141.000000,2.930000,904.000000,17.090000,0.000000,0.480000,1.440000,0.000000,0.000000,0.000000,13.730000
75%,2023-07-11 22:14:41,2023-07-11 22:17:23.500000,2023-07-11 22:19:03.500000,2023-07-11 22:44:39,211.000000,216.000000,5.960000,1417.000000,27.540000,0.000000,0.810000,2.370000,2.750000,0.000000,0.000000,22.470000
max,2024-08-31 21:35:47,2024-08-31 21:36:33,2024-08-31 21:37:05,2024-08-31 21:57:29,263.000000,265.000000,123.170000,9710.000000,347.900000,43.890000,10.440000,30.880000,2.750000,5.000000,58.790000,261.440000
std,NaN,NaN,NaN,NaN,75.543186,78.083027,5.684854,783.237487,19.468835,3.812899,0.627119,1.600158,1.323663,0.666302,2.920196,14.865439


### Processing Weather Data

In [381]:
def get_all_weather_csvs(directory):
    # reading all csv files in the directory
    files = glob.glob(os.path.join(directory,'*.csv'))
    
    csv_files =[]
    for file in files:
        # for each csv file, create a dataframe containing the data
        df = pd.read_csv(file, low_memory=False)
        
        csv_files.append(df)
    return csv_files

In [383]:
def clean_month_weather_data_hourly(csv_file):
    # find all column names in the data
    column_names = list(csv_file)
    # finding all column names containing 'Hourly', which refer to the hourly data
    keys = [name for name in column_names if "Hourly" in name]\
    # get the name of columns needed for hourly data
    columns = ["DATE"] + keys
    # create a dataframe containing the columns for hourly data
    hourly_data = csv_file[columns]
    # filter out non-hourly rows in the dataframe
    hourly_data = hourly_data[hourly_data["HourlyAltimeterSetting"].notna()]
    return hourly_data

In [385]:
def clean_month_weather_data_daily(csv_file):
    # find all column names in the data
    column_names = list(csv_file)
    # finding all column names containing 'Daily', which refer to the daily data
    keys = [name for name in column_names if "Daily" in name]
    # get the name of columns needed for daily data
    columns = ["DATE"] + keys
    # create a dataframe containing the columns for daily data
    daily_data = csv_file[columns]
    # filter out non-daily rows in the dataframe
    daily_data = daily_data[daily_data["DailyAverageDewPointTemperature"].notna()]
    return daily_data

In [436]:
def load_and_clean_weather_data():
    # get the dataframes of the csv files for weather data
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        # for each csv_file's data, clean and get the hourly and daily dataframes
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes containing the concatnetion of hourly & daily data respectively
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)

    # create hourly_range
    hourly_range = pd.date_range(start="2023-09-25 00:00:00", end="2023-10-03 23:59:59", freq="1H")
    hourly_range_df = pd.DataFrame(hourly_range, columns=["hour"])
    hourly_range_df["hour"] = hourly_range_df["hour"].astype(str)
    hourly_range_df.to_sql("hourly_range", con=engine, if_exists="replace", index=False) 
    
    return hourly_data, daily_data

In [438]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

/var/folders/kg/5js8flcn3c138fq2kf35s9vh0000gn/T/ipykernel_70862/4077312797.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_range = pd.date_range(start="2023-09-25 00:00:00", end="2023-10-03 23:59:59", freq="1H")


In [440]:
hourly_weather_data.head()

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySkyConditions,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2020-01-01T00:51:00,29.66,26,40,0.00,NaN,-0.01,3.0,58.0,BKN:07 80 OVC:08 100,29.64,29.49,10.00,35.0,VRB,NaN,8.0
1,2020-01-01T01:51:00,29.67,27,39,0.00,NaN,NaN,NaN,61.0,BKN:07 70 OVC:08 100,29.65,29.50,10.00,34.0,280,17.0,8.0
2,2020-01-01T02:51:00,29.68,26,39,0.00,NaN,NaN,NaN,60.0,FEW:02 70 OVC:08 90,29.66,29.51,10.00,34.0,260,23.0,14.0
3,2020-01-01T03:51:00,29.70,24,39,0.00,NaN,-0.03,3.0,55.0,OVC:08 75,29.67,29.53,10.00,33.0,250,23.0,11.0
4,2020-01-01T04:51:00,29.70,23,38,0.00,NaN,NaN,NaN,55.0,OVC:08 65,29.67,29.53,10.00,32.0,VRB,20.0,6.0


In [442]:
hourly_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54185 entries, 0 to 11636
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DATE                       54185 non-null  object 
 1   HourlyAltimeterSetting     54185 non-null  object 
 2   HourlyDewPointTemperature  54139 non-null  object 
 3   HourlyDryBulbTemperature   54179 non-null  object 
 4   HourlyPrecipitation        47495 non-null  object 
 5   HourlyPresentWeatherType   13987 non-null  object 
 6   HourlyPressureChange       13968 non-null  object 
 7   HourlyPressureTendency     13968 non-null  float64
 8   HourlyRelativeHumidity     54139 non-null  object 
 9   HourlySkyConditions        53612 non-null  object 
 10  HourlySeaLevelPressure     42023 non-null  object 
 11  HourlyStationPressure      53893 non-null  object 
 12  HourlyVisibility           54102 non-null  object 
 13  HourlyWetBulbTemperature   53847 non-null  object 


In [444]:
hourly_weather_data.describe()

,HourlyPressureTendency,HourlyWindGustSpeed,HourlyWindSpeed
count,13968.000000,8342.000000,49592.000000
mean,4.295604,21.579837,5.125302
std,2.791384,4.653003,14.662725
min,0.000000,16.000000,0.000000
25%,1.000000,18.000000,3.000000
50%,3.000000,21.000000,5.000000
75%,7.000000,24.000000,7.000000
max,9.000000,51.000000,2237.000000


In [446]:
daily_weather_data.head()

,DATE,DailyAverageDewPointTemperature,DailyAverageDryBulbTemperature,DailyAverageRelativeHumidity,DailyAverageSeaLevelPressure,DailyAverageStationPressure,DailyAverageWetBulbTemperature,DailyAverageWindSpeed,DailyCoolingDegreeDays,DailyDepartureFromNormalAverageTemperature,...,DailyMaximumDryBulbTemperature,DailyMinimumDryBulbTemperature,DailyPeakWindDirection,DailyPeakWindSpeed,DailyPrecipitation,DailySnowDepth,DailySnowfall,DailySustainedWindDirection,DailySustainedWindSpeed,DailyWeather
24,2020-01-01T23:59:00,21.0,38.0,52.0,29.76,29.62,32.0,8.6,0.0,4.6,...,41.0,34.0,260.0,29.0,0.00,0.0,0.0,270.0,17.0,NaN
49,2020-01-02T23:59:00,25.0,41.0,52.0,29.91,29.77,36.0,5.4,0.0,7.7,...,49.0,33.0,220.0,22.0,0.00,0.0,0.0,230.0,13.0,NaN
86,2020-01-03T23:59:00,41.0,47.0,82.0,29.81,29.67,44.0,3.4,0.0,13.9,...,49.0,44.0,230.0,15.0,0.15,0.0,0.0,250.0,10.0,RA BR HZ
144,2020-01-04T23:59:00,45.0,46.0,90.0,29.62,29.49,46.0,4.4,0.0,13.0,...,51.0,41.0,330.0,24.0,0.27,0.0,0.0,300.0,15.0,RA FG BR
169,2020-01-05T23:59:00,20.0,39.0,48.0,29.84,29.69,32.0,11.3,0.0,6.1,...,42.0,35.0,300.0,43.0,T,0.0,0.0,300.0,25.0,RA


In [448]:
daily_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1632 entries, 24 to 11637
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   DATE                                        1632 non-null   object 
 1   DailyAverageDewPointTemperature             1632 non-null   float64
 2   DailyAverageDryBulbTemperature              1632 non-null   float64
 3   DailyAverageRelativeHumidity                1632 non-null   float64
 4   DailyAverageSeaLevelPressure                1631 non-null   float64
 5   DailyAverageStationPressure                 1632 non-null   float64
 6   DailyAverageWetBulbTemperature              1632 non-null   float64
 7   DailyAverageWindSpeed                       1574 non-null   float64
 8   DailyCoolingDegreeDays                      1632 non-null   float64
 9   DailyDepartureFromNormalAverageTemperature  1632 non-null   float64
 10  DailyHeatingDeg

In [450]:
daily_weather_data.describe()

,DailyAverageDewPointTemperature,DailyAverageDryBulbTemperature,DailyAverageRelativeHumidity,DailyAverageSeaLevelPressure,DailyAverageStationPressure,DailyAverageWetBulbTemperature,DailyAverageWindSpeed,DailyCoolingDegreeDays,DailyDepartureFromNormalAverageTemperature,DailyHeatingDegreeDays,DailyMaximumDryBulbTemperature,DailyMinimumDryBulbTemperature
count,1632.000000,1632.000000,1632.000000,1631.000000,1632.000000,1632.000000,1574.000000,1632.000000,1632.000000,1632.000000,1632.000000,1632.000000
mean,42.547794,57.700980,61.020221,30.010846,29.863964,50.221814,5.048602,3.759191,2.554596,11.058211,64.518382,50.390319
std,18.201816,16.241926,15.695804,0.213623,0.214455,14.995885,2.360532,5.885186,6.896218,12.081967,17.109046,15.759504
min,-7.000000,11.000000,19.000000,29.220000,29.000000,8.000000,0.600000,0.000000,-24.200000,0.000000,15.000000,3.000000
25%,28.000000,45.000000,49.000000,29.875000,29.730000,38.000000,3.200000,0.000000,-2.000000,0.000000,50.000000,38.000000
50%,43.000000,58.000000,61.000000,30.000000,29.850000,50.000000,4.700000,0.000000,2.200000,7.000000,65.500000,50.000000
75%,58.000000,72.000000,72.000000,30.150000,30.002500,63.000000,6.500000,7.000000,7.000000,20.000000,79.000000,65.000000
max,74.000000,88.000000,95.000000,30.660000,30.510000,76.000000,14.200000,23.000000,28.800000,54.000000,98.000000,81.000000


## Part 2: Storing Cleaned Data

In [453]:
engine = db.create_engine(DATABASE_URL)

In [455]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    date_time TEXT PRIMARY KEY,
    temperature FLOAT,
    precipitation FLOAT,
    wind_speed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    date TEXT PRIMARY KEY,
    avg_temperature FLOAT,
    total_precipitation FLOAT,
    total_snowfall FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    trip_id INTEGER PRIMARY KEY,
    pickup_datetime TEXT,
    dropoff_datetime TEXT,
    trip_distance FLOAT,
    fare_amount FLOAT,
    tip_amount FLOAT,
    total_amount FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    trip_id INTEGER PRIMARY KEY,
    pickup_datetime TEXT,
    dropoff_datetime TEXT,
FLOAT    trip_distance FLOAT,
    fare_amount FLOAT,
    tip_amount FLOAT,
    total_amount FLOAT
);
"""

In [457]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [459]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

In [461]:
from sqlalchemy import inspect

inspector = inspect(engine)
print(inspector.get_table_names())

['daily_weather', 'hourly_range', 'hourly_weather', 'taxi_trips', 'uber_trips']


In [463]:
print(taxi_data.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


In [465]:
print(uber_data.columns)

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')


In [467]:
daily_weather_df = pd.read_sql("SELECT * FROM daily_weather LIMIT 1;", con=engine)
print(daily_weather_df.columns)

Index(['DATE', 'DailyAverageDewPointTemperature',
       'DailyAverageDryBulbTemperature', 'DailyAverageRelativeHumidity',
       'DailyAverageSeaLevelPressure', 'DailyAverageStationPressure',
       'DailyAverageWetBulbTemperature', 'DailyAverageWindSpeed',
       'DailyCoolingDegreeDays', 'DailyDepartureFromNormalAverageTemperature',
       'DailyHeatingDegreeDays', 'DailyMaximumDryBulbTemperature',
       'DailyMinimumDryBulbTemperature', 'DailyPeakWindDirection',
       'DailyPeakWindSpeed', 'DailyPrecipitation', 'DailySnowDepth',
       'DailySnowfall', 'DailySustainedWindDirection',
       'DailySustainedWindSpeed', 'DailyWeather'],
      dtype='object')


In [469]:
hourly_weather_df = pd.read_sql("SELECT * FROM hourly_weather LIMIT 1;", con=engine)
print(hourly_weather_df.columns)

Index(['DATE', 'HourlyAltimeterSetting', 'HourlyDewPointTemperature',
       'HourlyDryBulbTemperature', 'HourlyPrecipitation',
       'HourlyPresentWeatherType', 'HourlyPressureChange',
       'HourlyPressureTendency', 'HourlyRelativeHumidity',
       'HourlySkyConditions', 'HourlySeaLevelPressure',
       'HourlyStationPressure', 'HourlyVisibility', 'HourlyWetBulbTemperature',
       'HourlyWindDirection', 'HourlyWindGustSpeed', 'HourlyWindSpeed'],
      dtype='object')


In [473]:
from sqlalchemy import inspect

# 检查数据库中的表名
inspector = inspect(engine)
print(inspector.get_table_names())  # 确认是否有 'hourly_range'

['daily_weather', 'hourly_range', 'hourly_weather', 'taxi_trips', 'uber_trips']


In [475]:
def write_dataframes_to_table(table_to_df_dict):
    with engine.connect() as connection:
        for table_name, dataframe in table_to_df_dict.items():
            try:
                dataframe.to_sql(table_name, con=connection, if_exists="replace", index=False)
                print(f"Data written to table: {table_name}")
            except Exception as e:
                print(f"Failed to write to table {table_name}: {e}")

In [477]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [479]:
write_dataframes_to_table(map_table_name_to_dataframe)

Data written to table: taxi_trips
Data written to table: uber_trips
Data written to table: hourly_weather
Data written to table: daily_weather


## Part 3: Understanding the Data

In [481]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    filepath = os.path.join(QUERY_DIRECTORY, outfile)
    with open(filepath, "w") as f:
        f.write(query)
    print(f"Query written to file: {filepath}")

### Query 1

In [483]:
QUERY_1_FILENAME = "Query 1: Popular taxi hour.sql"

QUERY_1 = """
SELECT strftime('%H', tpep_pickup_datetime) AS hour,
       COUNT(*) AS ride_count
FROM taxi_trips
WHERE tpep_pickup_datetime BETWEEN '2020-01-01' AND '2024-08-31'
GROUP BY hour
ORDER BY ride_count DESC;
"""

In [484]:
# execute query either via pandas
df_query_1 = pd.read_sql(QUERY_1, con=engine)
print(df_query_1.head())

  hour  ride_count
0   18        1560
1   17        1497
2   15        1457
3   16        1410
4   14        1395


In [485]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

Query written to file: queries/Query 1: Popular taxi hour.sql


### Query 2

In [492]:
QUERY_2_FILENAME = "Query 2: Popular uber day.sql"

QUERY_2 = """
SELECT strftime('%w', pickup_datetime) AS weekday,
       COUNT(*) AS ride_count
FROM uber_trips
WHERE pickup_datetime BETWEEN '2020-01-01' AND '2024-08-31'
GROUP BY weekday
ORDER BY ride_count DESC;
"""

In [494]:
# execute query either via sqlalchemy
df_query_2 = pd.read_sql(QUERY_2, con=engine)
print(df_query_2.head())

  weekday  ride_count
0       6        2509
1       5        2352
2       4        2114
3       0        2027
4       3        1995


In [496]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

Query written to file: queries/Query 2: Popular uber day.sql


### Query 3

In [499]:
QUERY_3_FILENAME = "Query 3: Trip distance.sql"

QUERY_3 = """
WITH combined_trips AS (
    SELECT trip_distance
    FROM taxi_trips
    WHERE tpep_pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
    UNION ALL
    SELECT trip_miles AS trip_distance
    FROM uber_trips
    WHERE pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
),
sorted_trips AS (
    SELECT trip_distance
    FROM combined_trips
    ORDER BY trip_distance
),
percentile_index AS (
    SELECT CAST(COUNT(*) * 0.95 AS INTEGER) AS idx
    FROM sorted_trips
)
SELECT trip_distance AS percentile_95
FROM sorted_trips
LIMIT 1 OFFSET (SELECT idx - 1 FROM percentile_index);
"""

In [501]:
# execute query either via sqlalchemy
df_query_3 = pd.read_sql(QUERY_3, con=engine)
print(df_query_3.head())

   percentile_95
0           16.1


In [503]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

Query written to file: queries/Query 3: Trip distance.sql


### Query 4

In [506]:
QUERY_4_FILENAME = "Query 4: Busiest days weather.sql"

QUERY_4 = """
WITH daily_rides AS (
    SELECT strftime('%Y-%m-%d', pickup_datetime) AS ride_date,
           COUNT(*) AS total_rides,
           AVG(trip_distance) AS avg_distance
    FROM (
        SELECT tpep_pickup_datetime AS pickup_datetime, trip_distance
        FROM taxi_trips
        WHERE tpep_pickup_datetime BETWEEN '2023-01-01' AND '2023-12-31'
        UNION ALL
        SELECT pickup_datetime, trip_miles AS trip_distance
        FROM uber_trips
        WHERE pickup_datetime BETWEEN '2023-01-01' AND '2023-12-31'
    ) AS all_rides
    GROUP BY ride_date
)
SELECT d.ride_date, 
       d.total_rides, 
       d.avg_distance, 
       w.DailyPrecipitation AS avg_precipitation, 
       w.DailyAverageWindSpeed AS avg_wind_speed
FROM daily_rides d
LEFT JOIN daily_weather w ON d.ride_date = w.DATE
ORDER BY d.total_rides DESC
LIMIT 10;

"""

In [508]:
# execute query either via sqlalchemy
df_query_4 = pd.read_sql(QUERY_4, con=engine)
print(df_query_4.head())

    ride_date  total_rides  avg_distance avg_precipitation avg_wind_speed
0  2023-11-09           37      3.009459              None           None
1  2023-07-28           35      4.087429              None           None
2  2023-12-15           35      3.393429              None           None
3  2023-04-30           34      5.202647              None           None
4  2023-05-05           34      3.377647              None           None


In [510]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

Query written to file: queries/Query 4: Busiest days weather.sql


### Query 5

In [513]:
QUERY_5_FILENAME = "Query 5: Rides_on_snow_days.sql"

QUERY_5 = """
WITH daily_trip_counts AS (
    SELECT strftime('%Y-%m-%d', pickup_datetime) AS ride_date,
           COUNT(*) AS total_rides
    FROM (
        SELECT tpep_pickup_datetime AS pickup_datetime
        FROM taxi_trips
        WHERE tpep_pickup_datetime BETWEEN '2020-01-01' AND '2024-08-31'
        UNION ALL
        SELECT pickup_datetime
        FROM uber_trips
        WHERE pickup_datetime BETWEEN '2020-01-01' AND '2024-08-31'
    ) AS all_trips
    GROUP BY ride_date
)
SELECT w.DATE AS ride_date, 
       w.DailySnowfall AS total_snowfall, 
       COALESCE(d.total_rides, 0) AS total_rides
FROM daily_weather w
LEFT JOIN daily_trip_counts d ON w.DATE = d.ride_date
WHERE w.DATE BETWEEN '2020-01-01' AND '2024-08-31'
ORDER BY w.DailySnowfall DESC
LIMIT 10;

"""

In [515]:
df_query_5 = pd.read_sql(QUERY_5, con=engine)
print(df_query_5.head())

             ride_date total_snowfall  total_rides
0  2020-01-08T23:59:00              T            0
1  2020-01-16T23:59:00              T            0
2  2020-02-02T23:59:00              T            0
3  2020-02-07T23:59:00              T            0
4  2020-03-23T23:59:00              T            0


In [517]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

Query written to file: queries/Query 5: Rides_on_snow_days.sql


### Query 6

In [520]:
QUERY_6_FILENAME = "Query 6: Ophelia_weather_rides.sql"

QUERY_6 = """
WITH hourly_rides AS (
    SELECT strftime('%Y-%m-%d %H:00:00', pickup_datetime) AS hour,
           COUNT(*) AS total_rides
    FROM (
        SELECT tpep_pickup_datetime AS pickup_datetime
        FROM taxi_trips
        WHERE tpep_pickup_datetime BETWEEN '2023-09-25' AND '2023-10-03'
        UNION ALL
        SELECT pickup_datetime
        FROM uber_trips
        WHERE pickup_datetime BETWEEN '2023-09-25' AND '2023-10-03'
    ) AS all_trips
    GROUP BY hour
),
hourly_data AS (
    SELECT strftime('%Y-%m-%d %H:00:00', DATE) AS hour,
           SUM(HourlyPrecipitation) AS total_precipitation,
           AVG(HourlyWindSpeed) AS avg_wind_speed
    FROM hourly_weather
    WHERE DATE BETWEEN '2023-09-25 00:00:00' AND '2023-10-03 23:59:59'
    GROUP BY hour
)
SELECT hr.hour,
       COALESCE(r.total_rides, 0) AS total_rides,
       COALESCE(h.total_precipitation, 0) AS total_precipitation,
       COALESCE(h.avg_wind_speed, 0) AS avg_wind_speed
FROM hourly_range hr
LEFT JOIN hourly_rides r ON hr.hour = r.hour
LEFT JOIN hourly_data h ON hr.hour = h.hour
ORDER BY hr.hour ASC;
"""

In [522]:
# execute query either via sqlalchemy
df_query_6 = pd.read_sql(QUERY_6, con=engine)
print(df_query_6.head())

                  hour  total_rides  total_precipitation  avg_wind_speed
0  2023-09-25 00:00:00            0                 0.07        8.333333
1  2023-09-25 01:00:00            0                 0.12        7.000000
2  2023-09-25 02:00:00            0                 0.14        7.000000
3  2023-09-25 03:00:00            0                 0.04        7.000000
4  2023-09-25 04:00:00            0                 0.01        6.000000


In [524]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

Query written to file: queries/Query 6: Ophelia_weather_rides.sql


## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)